In [1]:
import pandas as pd
import numpy as np
import random
from simply import redshift
from scipy import stats

### STEP 1: Make a copy of this notebook in your own folder
####  - Preferably in the `src` folder associated with the ticket to pull the groups for the particular experiment.

### STEP 2: Generate the population from which to draw the samples
Enter your query below to generate a list of `user_ref`s from the population of interest. For this example I took all currently enrolled customers (removing known fraudsters). If there are any exclusionary criteria for your experiment, add those into this query. The result should be a list of all possible `user_ref`s from which the sample will be drawn. (**Note: Be cautious if adding exclusionary criteria - the results of the experiment can only be used to inform an understanding of the population of interest**). 

If you are interested in applying any insights/inferences to the entire customer base, you must draw from the entire customer base (i.e., no exclusionary criteria). If you are interested in understanding a subset of customers (e.g., people who live in cities)- then limit the population (e.g., list of all customers who live in cities), with the awareness that the results _cannot_ be used to make any inferences about customers not in the population (e.g., we will still know nothing about customers who do not live in cities after this experiment). 

Reach out if you have any questions!

In [2]:
df = redshift(
#Query here - I have used all currently enrolled customers who are not known fraudsters.  (Limited to enrolled in 2018 for query performance)
"""
SELECT user_ref 
FROM curated.dim_user
WHERE is_fraudster is false
    AND num_open_accounts > 0
    AND first_account_open_date BETWEEN '2018-01-01' AND '2018-12-31' --limited for query performance
"""
)

### STEP 3: Draw the random samples

Then we identify the size for the experimental and control groups. Often these are the same size, but this can vary. Note that if you want to use different sized experimental and control groups, the control group must _at minimum_ be big enough to have enough statistical power to detect a difference in the effect of interest. Experimental also needs to be big enough to detect a statistical difference. This can be calculated from a power analysis. Again, reach out if unsure. Once you have calculated the size for each group, enter them below.

In [3]:
# Modify the numbers here to reflect the number of customers you want to include in
#the experimental and control groups
experimental_num = 10000
control_num = 10000

Below we are generating the random samples for experimental and control group.

In [16]:
possible = pd.DataFrame(df.user_ref.unique())
possible.columns = ['user_ref']
experimental_users = random.sample(list(possible.user_ref), experimental_num)
remaining = possible[~possible.user_ref.isin(experimental_users)]
control_users = random.sample(list(remaining.user_ref), control_num)
unused = possible[(~possible.user_ref.isin(experimental_users))&(~possible.user_ref.isin(control_users))]

### STEP 4: Get the baseline metrics for experimental and control groups 

Now that we have our list of experimental and control users, we get their baseline metrics and compare to the total population. I have included average-balance-past-90-days, deposits-past-90-days, swipe-count-past-90-days, and swipe-volume-past-90-days. If you would like to include others add them to the query below (ex: KPIs or metrics you care about in the experiment). However, keep in mind that the more things you want to control for the more stringent your statistical test needs to be to avoid risk of a false positive.

**Rule of thumb: Focus on (ideally) 2-3 key metrics.** These will depend on the particular experiment and hypotheses. If you are conducting a similar test in two different populations (e.g., profitable versus non-profitable) - treat these as distinct experiments and focus on 2-3 key metrics in each (they can be the same or different metrics). For example, you might have a different hypothesis/assumption for profitable than for unprofitable customers; in this case your metrics of interest may differ.

In [5]:
baseline_query = """
SELECT 
    user_ref, 
    avg(balance_eod) AS avg_balance,
    sum(deposit_amount) AS deposits_past90,
    sum(swipe_amount) AS swipe_vol_past90,
    sum(swipe_count) AS swipe_count_past90
FROM curated.fact_customer_day
WHERE user_ref IN :users
    AND date > CURRENT_DATE-90
GROUP BY user_ref
"""

In [22]:
exp_ids = tuple(experimental_users)
cont_ids = tuple(control_users)
all_ids = tuple(possible.user_ref)

In [7]:
len(exp_ids)

10000

In [8]:
test = redshift(
"""
SELECT user_ref 
FROM curated.fact_customer_day
WHERE user_ref IN :ids
     AND date >= CURRENT_DATE-7
"""
, params = {'ids': exp_ids})

In [9]:
test.describe()

,user_ref
count,80000
unique,10000
top,b3a11722-529b-40ef-8d51-39bf0e87f238
freq,8


In [10]:
def get_activity(ids):
    return redshift(baseline_query, params = {'users':ids})

In [23]:
experimental_baseline = get_activity(exp_ids)
control_baseline = get_activity(cont_ids)
all_baseline = get_activity(all_ids)

In [24]:
all_baseline.describe()

,avg_balance,deposits_past90,swipe_vol_past90,swipe_count_past90
count,133631.000000,133631.000000,133631.000000,133631.000000
mean,525.983189,1299.312599,-703.805719,23.131639
std,4063.795985,4540.635011,2077.870132,58.332404
min,-9931.086000,0.000000,-91136.890000,0.000000
25%,0.000000,0.000000,-86.875000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,30.260500,178.000000,0.000000,5.000000
max,453461.877000,631295.170000,0.000000,805.000000


In [25]:
experimental_baseline.describe()

,avg_balance,deposits_past90,swipe_vol_past90,swipe_count_past90
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,505.398406,1241.779363,-686.862690,22.636300
std,3174.367129,3707.289965,1972.165563,57.038342
min,-1437.782000,0.000000,-43318.540000,0.000000
25%,0.000000,0.000000,-94.590000,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,28.772500,184.775000,0.000000,5.000000
max,111485.268000,105727.030000,0.000000,555.000000


In [26]:
control_baseline.describe()

,avg_balance,deposits_past90,swipe_vol_past90,swipe_count_past90
count,10000.000000,10000.000000,10000.000000,10000.000000
mean,506.422217,1306.618346,-691.431426,22.418950
std,3542.407041,4336.297753,2011.930129,56.916395
min,-1882.800000,0.000000,-35597.950000,0.000000
25%,0.000000,0.000000,-61.672500,0.000000
50%,0.000000,0.000000,0.000000,0.000000
75%,28.571250,150.000000,0.000000,4.000000
max,148731.683000,153362.295000,0.000000,577.000000


### STEP 5: Make sure the groups don't differ on contact rate

Note: You only need to run this part if you will be measuring the impact on contact rate in the experiment.

**THIS SECTION (QUERY, ETC.) NEEDS TO BE UPDATED ONCE CALL DATA IS ADDED TO REDSHIFT**

In [15]:
contact_rate_query = """

WITH population AS (
SELECT
	user_id 
FROM mtr_user_demographics
WHERE user_id NOT IN (select user_id FROM mtr_known_fraudsters)
    AND user_id in :ids
),

weekly_population AS (
SELECT DISTINCT
	user_id
FROM mtr_daily_customer_metrics
WHERE user_id IN (SELECT user_id FROM population)
	AND date > DATEADD(day, -30, GETDATE())
),

touches AS (
SELECT
  t.user_id,
  created_at,
  contact_type,
  thread_id,
  touch_id,
  ROW_NUMBER () OVER (PARTITION BY DATE_TRUNC('week', created_at), thread_id ORDER BY created_at) AS order_within_thread
FROM(
  SELECT
    CASE WHEN cc.user_id = '' OR cc.user_id IS NULL THEN 'call_id: ' + call_id ELSE cc.user_id END AS user_id,
    c.created_at,
    'call' AS contact_type,
    call_id AS thread_id,
    call_id AS touch_id
  FROM current_mabel_full_call c
    JOIN mtr_agent_phone_hours p ON DATE_TRUNC('hour', c.created_at) = p.hour
    LEFT JOIN mtr_customer_contact cc ON call_id = contact_uuid
    LEFT JOIN current_mabel_cdr x ON c.uniqueid = x.uniqueid
  WHERE COALESCE(c.duration, datediff(seconds, c.created_at, c.updated_at)) > 30
    AND c.src != '8882480632'
    AND (x.lastdata IS NULL OR x.lastdata NOT SIMILAR TO '%(apple|android)%')
    --AND (interface IS NOT NULL AND interface != '') -- answered calls ONLY, remove this line to include all calls
		AND cc.user_id IN (SELECT user_id FROM population)

  UNION ALL

  SELECT
    m.user_uuid AS user_id,
    m.created_at,
    'chat' AS contact_type,
    m.chat_uuid AS thread_id,
    m.uuid AS touch_id
  FROM current_hummingbird_messages m
  LEFT JOIN current_hummingbird_chats c on m.chat_uuid = c.uuid
  WHERE subject NOT IN('Apple Pay Verification', 'Android Pay Verification')
    AND "from" IS NULL
		AND m.user_uuid IN (SELECT user_id FROM population)
  ) t
JOIN population	USING(user_id)
LEFT JOIN mtr_user_first_activity USING(user_id)
LEFT JOIN mtr_monthly_customer_active_state s
  ON t.user_id = s.user_id
  AND DATEADD(month, -1, DATE_TRUNC('month', created_at)) = month
WHERE created_at > DATEADD(day, -30, GETDATE())
),

touches_by_customers AS(
SELECT
  user_id,
  SUM(CASE WHEN contact_type = 'call' THEN 1 ELSE 0 END) AS count_calls,
  SUM(CASE WHEN contact_type = 'chat' AND order_within_thread = 1 THEN 1 ELSE 0 END) AS count_chats,
  SUM(CASE WHEN contact_type = 'chat' THEN 1 ELSE 0 END) AS count_messages,
  SUM(CASE WHEN contact_type = 'chat' THEN 1 ELSE NULL END) / SUM(CASE WHEN contact_type = 'chat' AND order_within_thread = 1 THEN 1 ELSE NULL END) AS messages_per_chat
FROM touches
WHERE created_at > DATEADD(day, -30, GETDATE())
GROUP BY 1
),

total_contact_by_user_by_week AS (
SELECT
	user_id,
	SUM(COALESCE(count_calls, 0) + COALESCE(count_messages, 0)) AS total_contact
FROM touches_by_customers	
GROUP BY
	1
)

SELECT DISTINCT
	n.user_id,
	CASE WHEN total_contact IS NULL THEN 0 ELSE 1 END AS unique_contact,
	CASE WHEN total_contact IS NULL THEN 0 ELSE total_contact END AS total_contact
FROM weekly_population	n
LEFT JOIN total_contact_by_user_by_week	w	on n.user_id = w.user_id
ORDER BY 1

"""

In [16]:
exp_contact = redshift(contact_rate_query, params = {'ids':exp_ids})

In [17]:
con_contact = redshift(contact_rate_query, params = {'ids':cont_ids})

In [18]:
exp_contact.describe()

,unique_contact,total_contact
count,15990.000000,15990.000000
mean,0.074171,0.171545
std,0.262058,0.850088
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,1.000000,23.000000


In [19]:
con_contact.describe()

,unique_contact,total_contact
count,15994.000000,15994.000000
mean,0.071777,0.173753
std,0.258126,0.903045
min,0.000000,0.000000
25%,0.000000,0.000000
50%,0.000000,0.000000
75%,0.000000,0.000000
max,1.000000,28.000000


In [20]:
# Calculation Unique Contact - Chi Square Test
# Note: the Numpy array needs to be 2-dimensional! 
obs = np.array([[exp_contact[exp_contact.unique_contact ==1].user_id.count(),
                 exp_contact[exp_contact.unique_contact ==0].user_id.count()],
                [con_contact[con_contact.unique_contact ==1].user_id.count(),
                 con_contact[con_contact.unique_contact ==0].user_id.count()]])

chi2, p, dof, expected = stats.chi2_contingency(obs)

#### This p-value indicates whether there's a statistial difference in the proportion of people who contact in each group (experimental versus control).
Re-pull the groups if the p < .1

In [23]:
#P Value for the Chi Square Test
p

1.0

In [24]:
#Returns P Value for Total Contact Rate
[tstat,pvalue] = stats.ttest_ind(exp_contact.total_contact, con_contact.total_contact)

#### This p-value indicates whether there's a statistical difference in the total contact between the groups (experimental versus control)
Re-pull the groups if the pvalue <.1

In [25]:
pvalue

0.82187717804278693

### Step 6: Conduct statistical tests to ensure that experimental and control groups are statistically indistinguishable from each other and from the population (on the metrics we are interested in - other than contact)

Here we will conduct independent t-tests for each measure to investigate baseline differences between (1) experimental group and all customers and (2) experimental and control groups. We are looking for high p-values here to indicate no difference. (If the p-values are trending towards significance there is a problem.) 

**Note that it is important to find statistical similarity (e.g., p > .2) in the variables of interest.** For example, if you are going to be testing for an increase in ADB, then it will be important that the experiment and control group have similar ADB at the beginning of the experiment. If you run the t-test and you find that the groups have different mean ADB, and the p-value is trending towards signfiicance then I would re-pull the groups (e.g., may not be lower than .05 but .1 is still too low; this means that theres only a 10% chance that the mean differences would have been found by chance). Feel free to rerun the notebook until you get similar means and high p-values. 

In [27]:
print('Comparing experimental to total population')
print('-------------------------------------------')
print('Average balance:',stats.ttest_ind(experimental_baseline.avg_balance, all_baseline.avg_balance))
print('Deposits:',stats.ttest_ind(experimental_baseline.deposits_past90, all_baseline.deposits_past90))
print('Swipe volume:',stats.ttest_ind(experimental_baseline.swipe_vol_past90, all_baseline.swipe_vol_past90))
print('Swipe count:',stats.ttest_ind(experimental_baseline.swipe_count_past90, all_baseline.swipe_count_past90))
print('----------------------------------------------------')
print('----------------------------------------------------')
print('Comparing experimental to control')
print('-------------------------------------------')
print('Average balance:',stats.ttest_ind(experimental_baseline.avg_balance, control_baseline.avg_balance))
print('Deposits:',stats.ttest_ind(experimental_baseline.deposits_past90, control_baseline.deposits_past90))
print('Swipe volume:',stats.ttest_ind(experimental_baseline.swipe_vol_past90, control_baseline.swipe_vol_past90))
print('Swipe count:',stats.ttest_ind(experimental_baseline.swipe_count_past90, control_baseline.swipe_count_past90))

Comparing experimental to total population
-------------------------------------------
Average balance: Ttest_indResult(statistic=-0.49535718242675125, pvalue=0.620348771147617)
Deposits: Ttest_indResult(statistic=-1.236604533872439, pvalue=0.2162359579300193)
Swipe volume: Ttest_indResult(statistic=0.7892349217876605, pvalue=0.4299760175877353)
Swipe count: Ttest_indResult(statistic=-0.820325346354133, pvalue=0.41203202220384727)
----------------------------------------------------
----------------------------------------------------
Comparing experimental to control
-------------------------------------------
Average balance: Ttest_indResult(statistic=-0.02152399522123317, pvalue=0.9828278772172596)
Deposits: Ttest_indResult(statistic=-1.1365211601971195, pvalue=0.25575213347159104)
Swipe volume: Ttest_indResult(statistic=0.16216591421043977, pvalue=0.8711768393619945)
Swipe count: Ttest_indResult(statistic=0.26973792421604015, pvalue=0.7873646654139164)


### STEP 7: Save the experimental and control user_refs as CSV files

Save the csv files in the `output` folder (presuming you are currently working in the `src` folder). You can then link to those csv files in the ticket.

In [30]:
experimental_baseline.user_ref.to_csv('../output/experimental_ids.csv', header=True, index=False)
control_baseline.user_ref.to_csv('../output/control_ids.csv', header=True, index=False)